In [1]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from link_builder import get_rec_link
from recmegpt_module import RecMeGPT

<h3 >Welcome to RecMeGPT! </h3>
Discover your next obsession with RecMeGPT! Unleash a world of entertainment tailored just for you! From mind-bending albums to immersive games, cinematic gems, gripping TV shows, and literary masterpieces. Explore, experience, and elevate your entertainment journey with our curated suggestions. Your next favorite thing awaits!<br>
You can enter anything from any media you like and even love! From a theme to a song or band or movie and anything in between! We will do our best to get you going!

In [2]:
# The textbox for the user to input what they need and the button to call the GPT Model and show the results
user_input = widgets.Text( value='', placeholder='Type something you like!', description='Your input:')
gpt_button = widgets.Button(description='Get your recs!', tooltip='Click!')
# Output to show the recommendations
output_area = widgets.Output()

def get_recommendations(a):
    """
    Calls the class RecMeGPT to try to get the recommendation based on the textbox.
    If successful, it will the display them. If not, it will tell the user it couldn't be done and to try again.
    Args:
    - a (widgets.Button): The input by the user representing what they like and want to get recommendations based off.    
    Returns:
    - None
    """
         
    def display_recs(recs,header,index):
        """
        Inner function to display the recommendations in HTML tags
         Args:
        - recs (list): List of dictionaries with the recommendations for a specific media
        - header (str): Value of the HTML header
        - index (int):  Numeric identifier of the media, used to create hyperlinks for each recommendation  
        Returns:
        - None
        """
        displayed_text=f"<h2>{header}</h2>"
        for rec in recs:
            displayed_text += f'<h3>{rec["title"]}</h3>{rec["description"]} - {get_rec_link(rec["title"],index)} <br>'
        with output_area:
            display(HTML(displayed_text))

    try:
        # Disables interactive objects so the user cannot send a new request if one is already running        
        user_input.disabled = True
        gpt_button.disabled = True
                
        # Checks if the input was empty, if so, it will warn the user to fix it
        if not user_input.value:
            raise Exception("You forgot to input something! Please try again!")

        # Checks if the input is too long, if so, it asks the user to shorten it
        if len(user_input.value) > 100:
            raise Exception("Try to keep your input less than 100 characters long! Please try again!")
            
        # Starts looking for the recommendations
        with output_area:
            clear_output(wait=True)
            display(HTML("<h3>Looking for recommendations...</h3>"))
        
        # Creates an object of the class in charge of interacting with chatGPT
        recme_gpt = RecMeGPT()
        # Sets the response to data, depending on the GPT model reply
        data = recme_gpt.request_recommendation(user_input.value.strip(),1)
        
        # If there was an exception on the request, it warns the user
        if "exception" in data:
            raise Exception(data["exception"])
        
        # If the data was OK (reply from GPT with recommendations) 
        # it will show the data of the recommendations
        if data["status"] == "OK":            
            # In case the recommendations don't come in order, this dictionary will provide 
            # the indentifier of media for the link builder and the html header
            html_display = {
                "albums": [0,"Music Albums"],
                "games": [1,"Videogames"],             
                "movies": [2,"Movies"],
                "tv": [3,"TV Shows"],
                "books": [4,"Books"]
            }
            
            with output_area:
                display(HTML(f"<h1>Your recommendations!</h1>{data['header']}" ))                
                       
            for media, recs in data["recs"].items():
                display_recs(recs, html_display[media][1], html_display[media][0])
                
        #if there no problems connecting to the GPT Model, but the model couldn't find anything...
        # it warns the user to try again
        else:
            with output_area:
                clear_output(wait=True)                
                display(HTML("<h2>Oops!</h2>I could not find anything, sorry! Please try again!" ))
            
    except Exception as e:
        error = str(e)     
        with output_area:
            clear_output(wait=True)
            display(HTML(f"<h2>Oops!</h2>{error}"))
    finally:
        # Re-enables the interactive objects, so the user can try again
        user_input.disabled = False
        gpt_button.disabled = False

# Set the function to run when the button is clicked
gpt_button.on_click(get_recommendations)
# Displays the input textbox, button to get the recommendations and error messages/recommendations
display(user_input, gpt_button)
display(output_area)


Text(value='', description='Your input:', placeholder='Type something you like!')

Button(description='Get your recs!', style=ButtonStyle(), tooltip='Click!')

Output()